In [2]:
!pip install --upgrade git+https://github.com/flairNLP/flair.git

  Cloning https://github.com/flairNLP/flair.git to /tmp/pip-req-build-2uq1kiol
  Running command git clone -q https://github.com/flairNLP/flair.git /tmp/pip-req-build-2uq1kiol
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 798kB 6.5MB/s 
     |████████████████████████████████| 645kB 36.3MB/s 
     |████████████████████████████████| 983kB 39.0MB/s 
     |████████████████████████████████| 256kB 36.7MB/s 
     |████████████████████████████████| 3.8MB 19.2MB/s 
     |████████████████████████████████| 1.1MB 34.5MB/s 
     |████████████████████████████████| 890kB 38.6MB/s 
  Created wheel for flair: filename=flair-0.4.5-cp36-none-any.whl size=148505 sha256=0e830f8bcfe065dc0e53bb3c8e6ff68448d9dd279dd29ca510a10f7f781abba7
  Stored in directory: /tmp/pip-ephem-wheel-cache-i8wc5kpo/wheels/84/82/73/d2b3b59b7be74ea05f2c6d64132efe27df52daffb47d1dc7bb
Successfully built flair
  Crea

In [0]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings, DocumentRNNEmbeddings, BertEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
from flair.datasets import CSVClassificationCorpus
from flair.data import Corpus
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data_folder = "./drive/My Drive/capstone/data/"

### First Stage (Train on benchmark dataset)

In [0]:
benchmark = pd.read_csv(data_folder + "combined_benchmark.csv")

In [10]:
benchmark = benchmark[['label', 'text']]
benchmark.head()

,label,text
0,0,Why not subscribe to the magazine ?
1,-1,"Tornio Works employs 2,300 of whom more than 1..."
2,1,"The move is aimed at boosting sales , cost-eff..."
3,0,"As a result of the merger , the largest profes..."
4,-1,18 March 2010 A leakage in the gypsum pond was...


#### Create train, dev and test set

In [0]:
benchmark = benchmark.sample(frac=1)
benchmark.iloc[0:int(len(benchmark)*0.8)].to_csv(data_folder + 'train.csv', sep='\t', index = False, header = False)
benchmark.iloc[int(len(benchmark)*0.8):int(len(benchmark)*0.9)].to_csv(data_folder + 'test.csv', sep='\t', index = False, header = False)
benchmark.iloc[int(len(benchmark)*0.9):].to_csv(data_folder + 'dev.csv', sep='\t', index = False, header = False)

#### Build corpus

In [12]:
# corpus = NLPTaskDataFetcher.load_classification_corpus(Path(data_folder), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')
column_name_map = {1: "text", 0: "label_topic"}

corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         column_name_map,
                                         skip_header=False, #no header in kaggle data
                                         delimiter='\t',    # comma separated rows
)

2020-05-21 21:33:57,559 Reading data from drive/My Drive/capstone/data
2020-05-21 21:33:57,560 Train: drive/My Drive/capstone/data/train.csv
2020-05-21 21:33:57,561 Dev: drive/My Drive/capstone/data/dev.csv
2020-05-21 21:33:57,563 Test: drive/My Drive/capstone/data/test.csv


#### Create word embeddings

In [13]:
word_embeddings = [BertEmbeddings(), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method __init__. (Use 'TransformerWordEmbeddings' for all transformer-based word embeddings) -- Deprecated since version 0.4.5.
  """Entry point for launching an IPython kernel.



2020-05-21 21:34:20,508 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-forward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmpx661a1f1


100%|██████████| 19689779/19689779 [00:00<00:00, 49444260.47B/s]

2020-05-21 21:34:21,096 copying /tmp/tmpx661a1f1 to cache at /root/.flair/embeddings/lm-news-english-forward-1024-v0.2rc.pt


2020-05-21 21:34:21,125 removing temp file /tmp/tmpx661a1f1
2020-05-21 21:34:37,776 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-backward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmp119mrbt1


100%|██████████| 19689779/19689779 [00:00<00:00, 51132721.58B/s]

2020-05-21 21:34:38,322 copying /tmp/tmp119mrbt1 to cache at /root/.flair/embeddings/lm-news-english-backward-1024-v0.2rc.pt


2020-05-21 21:34:38,354 removing temp file /tmp/tmp119mrbt1


#### First Stage Fine-tuning

In [14]:
document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)
trainer = ModelTrainer(classifier, corpus)
trainer.train(data_folder, max_epochs=10)

2020-05-21 21:34:44,497 Computing label dictionary. Progress:


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method __init__. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  """Entry point for launching an IPython kernel.
100%|██████████| 1314/1314 [00:02<00:00, 642.58it/s]

2020-05-21 21:34:46,855 [b'0', b'-1', b'1']
2020-05-21 21:34:46,874 ----------------------------------------------------------------------------------------------------
2020-05-21 21:34:46,877 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BertEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=Tru

2020-05-21 21:34:49,614 epoch 1 - iter 3/37 - loss 1.37792166 - samples/sec: 41.88
2020-05-21 21:35:04,758 epoch 1 - iter 6/37 - loss 1.38339380 - samples/sec: 50.24
2020-05-21 21:35:20,032 epoch 1 - iter 9/37 - loss 1.25956689 - samples/sec: 47.84
2020-05-21 21:35:35,205 epoch 1 - iter 12/37 - loss 1.27234081 - samples/sec: 44.19
2020-05-21 21:35:50,097 epoch 1 - iter 15/37 - loss 1.23371030 - samples/sec: 56.95
2020-05-21 21:36:05,057 epoch 1 - iter 18/37 - loss 1.22769420 - samples/sec: 57.27
2020-05-21 21:36:19,920 epoch 1 - iter 21/37 - loss 1.21607478 - samples/sec: 57.71
2020-05-21 21:36:35,063 epoch 1 - iter 24/37 - loss 1.20891273 - samples/sec: 52.59
2020-05-21 21:36:50,015 epoch 1 - iter 27/37 - loss 1.19474173 - samples/sec: 55.33
2020-05-21 21:37:04,955 epoch 1 - iter 30/37 - loss 1.17071977 - samples/sec: 52.95
2020-05-21 21:37:19,909 epoch 1 - iter 33/37 - loss 1.14583929 - samples/sec: 54.74
2020-05-21 21:37:34,727 epoch 1 - iter 36/37 - loss 1.15452903 - samples/sec: 5

{'dev_loss_history': [0.9445202946662903,
  0.931380569934845,
  1.0202172994613647,
  0.8029612898826599,
  0.8560692667961121,
  0.8320075869560242,
  1.0983926057815552,
  0.8370154500007629,
  1.3560564517974854,
  0.8271321654319763],
 'dev_score_history': [0.7006802721088435,
  0.6870748299319728,
  0.6961451247165533,
  0.7414965986394558,
  0.7596371882086168,
  0.7324263038548753,
  0.7052154195011338,
  0.746031746031746,
  0.6870748299319728,
  0.7732426303854876],
 'test_score': 0.776255707762557,
 'train_loss_history': [1.1594745307355314,
  0.9738980918317228,
  0.8259133777102908,
  0.7832235951681394,
  0.7250221081682153,
  0.703832321070336,
  0.6765379398255735,
  0.5982233124810297,
  0.632674384761501,
  0.4617382706822576]}

### Second Stage (train on hand annotated datasets)

#### Build corpus

In [9]:
new_data_folder = './drive/My Drive/capstone/data/second_stage/'
new_column_name_map = {5: "text", 4: "label_topic"}

corpus: Corpus = CSVClassificationCorpus(new_data_folder,
                                         new_column_name_map,
                                         skip_header=True, #no header in kaggle data
                                         delimiter=',',    # comma separated rows
)

2020-05-21 22:56:20,848 Reading data from drive/My Drive/capstone/data/second_stage
2020-05-21 22:56:20,850 Train: drive/My Drive/capstone/data/second_stage/train.csv
2020-05-21 22:56:20,852 Dev: drive/My Drive/capstone/data/second_stage/dev.csv
2020-05-21 22:56:20,858 Test: drive/My Drive/capstone/data/second_stage/test.csv


#### Second Stage fine-tuning

In [10]:
# benchmark_embeddings = FlairEmbeddings(data_folder + 'best-model.pt')
benchmark_classifier = TextClassifier.load(data_folder + 'best-model.pt')

2020-05-21 22:56:22,462 loading file ./drive/My Drive/capstone/data/best-model.pt


In [11]:
trainer = ModelTrainer(benchmark_classifier, corpus)
trainer.train(new_data_folder, max_epochs=10)

2020-05-21 22:56:25,413 ----------------------------------------------------------------------------------------------------
2020-05-21 22:56:25,418 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BertEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Linear(in_feat

{'dev_loss_history': [0.6211347579956055,
  0.6653227806091309,
  0.5546189546585083,
  0.4246145486831665,
  0.34248945116996765,
  2.3637852668762207,
  0.3363116681575775,
  1.2868273258209229,
  0.8674789667129517,
  0.4017622470855713],
 'dev_score_history': [0.7777777777777778,
  0.7777777777777778,
  0.7777777777777778,
  0.8383838383838383,
  0.8787878787878788,
  0.6161616161616161,
  0.8787878787878788,
  0.7171717171717171,
  0.8181818181818182,
  0.898989898989899],
 'test_score': 0.898989898989899,
 'train_loss_history': [1.22447270154953,
  1.0648577809333801,
  0.7276832222938537,
  0.6568765461444854,
  0.4097093641757965,
  0.4592723071575165,
  0.9268317580223083,
  0.26085380017757415,
  0.3351425021886826,
  0.37118481993675234]}